# Library

In [2]:
#import sys
import pandas as pd
import numpy as np
import random
import pyspark
import itertools
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import pickle
import statsmodels.api as sm

from pyspark import SparkContext, SQLContext

from math import sqrt
from time import time as ttt

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier as DTC_spark
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import joblib
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import parallel_backend


# Upload

In [2]:
spark = SparkSession.builder.master("local").appName("spark_app_1234").getOrCreate()
sc = spark.sparkContext
spark
d0 = (
    spark
    .read
    .format("csv") 
    .option("header","true") 
    .option("inferSchema","true") 
    .load("gs://mas-a5-storage-1/notebooks/jupyter/application_train.csv")
)

d1 = d0.filter(d0.DAYS_EMPLOYED != 365243).select('TARGET','DAYS_EMPLOYED')

print(d1.rdd.getNumPartitions())

4


# Function

In [3]:
def prepare_spark_data(n_part, k_mult):
    '''
    takes 'DAYS_EMPLOYED and 'TARGET'  from d1 (alreay filtered)
    rearrane in n partitions (if n==0 keeps initial number of partitions)
    prints final shape/ partition
    returns d2 - spark df
    '''
    data = d1
    data_new = data # first step in the cycle
    for i in range(k_mult-1):
        data_tmp = data.select('TARGET', \
                     f.col('DAYS_EMPLOYED')*(f.lit(0.9995) + f.rand()/1000)).\
                   toDF('TARGET','DAYS_EMPLOYED')
        data_tmp = data_tmp.select('TARGET', f.floor('DAYS_EMPLOYED'))
        data_new = data_new.union(data_tmp)

    assembler = VectorAssembler(inputCols=["DAYS_EMPLOYED"], 
                        outputCol="DAYS_EMPLOYED_vect")
    d2 = assembler.transform(data_new)
    if n_part != 0:
        d2 = d2.repartition(n_part)       
    #print(f'n-partitions initial: {d2.rdd.getNumPartitions()}; df size: {d2.count()}\n')

    return d2

# Params an run

In [ ]:
fn = '4cpu_by_6n__4m'
size_mult = [1, 2, 5, 10, 15, 20, 30, 50]
partitions = [0, 2, 4, 6, 8, 16, 24] 
# initial, 2**i incl n_nodes till n_cpu*n_nodes and n_cpu*n_nodes
n_iter = 3

print('d1-size', d1.count())
n_part_base = d1.rdd.getNumPartitions()
print('n-partitions initial', n_part_base, '\n')

rd1 = {}
for k_size_mult in size_mult:
    rd2 = {}
    for n_part in partitions:
        df = prepare_spark_data(n_part, k_size_mult)
        df.cache()
        print('======================================================================')
        print(f'=== size_mult={k_size_mult}; \
        n_part_req={n_part}; n_part_act={df.rdd.getNumPartitions()}; df_count={df.count()} ===\n')
        times = [0 for i in range(n_iter)]
        models = {}
        for i in range(n_iter):
            dt = DTC_spark(labelCol="TARGET",
                       featuresCol="DAYS_EMPLOYED_vect",
                       minInfoGain=0.0001,
                       impurity='entropy',
                       maxDepth=14, maxBins=2**14, # it differs from scikit learn - it means number of canidate split points
                       #minInstancesPerNode = 1,
                       #checkpointInterval = 10
                       )
           
            t0 = ttt()
            model = dt.fit(df)
            t1 = ttt()
            times[i] = t1-t0
            models[i] = model 
            print('model', i, 'build time', round(times[i],2), '\n', model)
        rd2[n_part] = (models, times)
    rd1[k_size_mult] = rd2

result = rd1  

d1-size 252137
n-partitions initial 4 



=== size_mult=1;         n_part_req=0; n_part_act=14; df_count=252137 ===

model 0 build time 6.19 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_5d1823802f8f, depth=14, numNodes=123, numClasses=2, numFeatures=1
model 1 build time 4.72 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_158adef97fe0, depth=14, numNodes=123, numClasses=2, numFeatures=1
model 2 build time 4.65 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_4723484652af, depth=14, numNodes=123, numClasses=2, numFeatures=1
=== size_mult=1;         n_part_req=2; n_part_act=2; df_count=252137 ===



model 0 build time 7.36 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_e21e7f3b1495, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 1 build time 7.16 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_13872054975f, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 2 build time 6.85 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_3d6639470fe1, depth=14, numNodes=123, numClasses=2, numFeatures=1
=== size_mult=1;         n_part_req=4; n_part_act=4; df_count=252137 ===



...

model 2 build time 28.88 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_346e14e82005, depth=13, numNodes=47, numClasses=2, numFeatures=1


=== size_mult=50;         n_part_req=24; n_part_act=24; df_count=12606850 ===



model 0 build time 30.88 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_9fe0dd9fa96f, depth=13, numNodes=47, numClasses=2, numFeatures=1


model 1 build time 31.93 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_3aede4b01492, depth=13, numNodes=47, numClasses=2, numFeatures=1


model 2 build time 32.91 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_d4ab2ff1087c, depth=13, numNodes=47, numClasses=2, numFeatures=1


# Results

In [7]:
di = result
df_res_time = pd.DataFrame()
df_res_nodes = pd.DataFrame()
for p1 in di.keys():
    for p2 in di[p1].keys():
        df_res_time.loc[p1, p2] = np.round(np.mean(di[p1][p2][1]), 1)
        m_tmp = di[p1][p2][0]
        n_nodes = []
        for i,_ in enumerate(m_tmp):
            tmp = f'{m_tmp[i]}'.split(' ')
            print(i, tmp)
            n_nodes.append([int(x.split('=')[1][:-1]) for x in tmp if x[:4]== 'numN'][0])
        df_res_nodes.loc[p1, p2] = np.round(np.mean(n_nodes), 1)

df_res_time.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
df_res_nodes.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')

display(df_res_time)      
display(df_res_nodes)     

0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_5d1823802f8f,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_158adef97fe0,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_4723484652af,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_e21e7f3b1495,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_13872054975f,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_3d6639470fe1,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_96b9486294a7,', 'depth=14,', 'numNodes=123,', 'numC

,0,2,4,6,8,16,24
1,5.2,7.1,5.7,5.5,5.5,8.5,9.8
2,7.0,9.5,6.2,8.0,8.0,11.5,14.4
5,17.4,14.3,9.4,9.6,10.2,13.9,16.9
10,39.0,18.1,11.8,16.0,19.6,26.2,32.5
15,69.8,24.6,14.3,20.0,25.7,32.6,39.8
20,96.5,27.7,15.9,16.2,15.6,19.3,23.1
30,128.2,32.1,18.3,23.9,31.2,39.2,46.6
50,214.4,48.4,26.0,27.4,25.2,28.6,31.9


,0,2,4,6,8,16,24
1,123.0,123.0,123.0,123.0,123.0,123.0,123.0
2,151.0,151.0,151.0,151.0,151.0,151.0,151.0
5,67.0,67.0,67.0,67.0,67.0,67.0,67.0
10,65.0,65.0,65.0,65.0,65.0,65.0,65.0
15,35.0,35.0,35.0,35.0,35.0,35.0,35.0
20,57.0,57.0,57.0,57.0,57.0,57.0,57.0
30,41.0,41.0,41.0,41.0,41.0,41.0,41.0
50,47.0,47.0,47.0,47.0,47.0,47.0,47.0


In [3]:
fn = '4cpu_by_6n__4m'

tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
tmp

,Unnamed: 0,0,2,4,6,8,16,24
0,1,5.2,7.1,5.7,5.5,5.5,8.5,9.8
1,2,7.0,9.5,6.2,8.0,8.0,11.5,14.4
2,5,17.4,14.3,9.4,9.6,10.2,13.9,16.9
3,10,39.0,18.1,11.8,16.0,19.6,26.2,32.5
4,15,69.8,24.6,14.3,20.0,25.7,32.6,39.8
5,20,96.5,27.7,15.9,16.2,15.6,19.3,23.1
6,30,128.2,32.1,18.3,23.9,31.2,39.2,46.6
7,50,214.4,48.4,26.0,27.4,25.2,28.6,31.9


In [4]:
fn = '4cpu_by_6n__4m'
tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')
tmp

,Unnamed: 0,0,2,4,6,8,16,24
0,1,123.0,123.0,123.0,123.0,123.0,123.0,123.0
1,2,151.0,151.0,151.0,151.0,151.0,151.0,151.0
2,5,67.0,67.0,67.0,67.0,67.0,67.0,67.0
3,10,65.0,65.0,65.0,65.0,65.0,65.0,65.0
4,15,35.0,35.0,35.0,35.0,35.0,35.0,35.0
5,20,57.0,57.0,57.0,57.0,57.0,57.0,57.0
6,30,41.0,41.0,41.0,41.0,41.0,41.0,41.0
7,50,47.0,47.0,47.0,47.0,47.0,47.0,47.0
